In [1]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [2]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [3]:
# 读取数据
data = pd.read_excel('data.xlsx')  # 默认header=0，第1行

# 构建特征和标签集合
y = data['y'].values  # 定义标签
X = data.drop(['y'], axis=1).values  # 定义新的特征集

# 转换为张量
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# 创建数据集和数据加载器
dataset = CustomDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# 划分训练集、测试集
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
another_train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [4]:
# 定义神经网络模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(9, 64)
        self.fc2 = nn.Linear(64, 36)  # 修正层大小
        self.fc3 = nn.Linear(36, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 创建模型实例
model = NeuralNetwork()

In [5]:
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [6]:
# 训练模型
total_loss = 0
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss
    print(loss)

In [7]:
import matplotlib.pyplot as plt
model.eval()
val_loss = 0.0
# 修正：x_batch 应为 x, 修正为合适的输入维度
for x_batch, y_batch in test_loader:
    outputs = model(x_batch)
    plt.scatter(outputs.detach().numpy(), y_batch)
plt.show()